In [1]:
import pandas as pd
import numpy as np
import functools as ft

In [2]:
def identify_reservations(df):
    try:
        Fab300_raw_reservations = df.copy()
        Fab300_raw_reservations["DATE_TIME_STAMP"] = pd.to_datetime(Fab300_raw_reservations["DATE_TIME_STAMP"])
        SortedRows = Fab300_raw_reservations.sort_values(["EVENT_ROW_ID"])

        index = range(1,len(SortedRows)+1)
        IndexShift_1 = [i-1 if i%2 != 0 else np.nan for i in index]
        IndexShift_2 = [i-1 if i%2 == 0 else np.nan for i in index]

        SortedRows["Index"] = index
        SortedRows["IndexShift_1"] = IndexShift_1
        SortedRows["IndexShift_2"] = IndexShift_2
        SortedRows["IndexShift_1_1"] = SortedRows["IndexShift_1"].fillna(method='bfill')
        SortedRows["IndexShift_2_1"] = SortedRows["IndexShift_2"].fillna(method='bfill')
        SortedRows = SortedRows.drop(columns=['IndexShift_1', 'IndexShift_2'])
        SortedRows = SortedRows.rename(columns={"IndexShift_1_1":"IndexShift_1","IndexShift_2_1":"IndexShift_2"})
        FilledUp2 = SortedRows.copy()

        UnpivotedOnlySelectedColumns = pd.melt(FilledUp2, id_vars=['FO_ROW_ID','Index','IndexShift_1','IndexShift_2'], 
                    value_vars=["ResWBS", "ResTk", "DATE_TIME_STAMP", "USER_ID", "EVENT_ROW_ID"])

        UnpivotedOnlySelectedColumns["ResProperty_1"] = np.where(UnpivotedOnlySelectedColumns['Index']==UnpivotedOnlySelectedColumns['IndexShift_1'], UnpivotedOnlySelectedColumns['variable'] + "_Start", UnpivotedOnlySelectedColumns['variable'] + "_End")
        UnpivotedOnlySelectedColumns["ResProperty_2"] = np.where(UnpivotedOnlySelectedColumns['Index']==UnpivotedOnlySelectedColumns['IndexShift_2'], UnpivotedOnlySelectedColumns['variable'] + "_Start", UnpivotedOnlySelectedColumns['variable'] + "_End")

        AddedCustom4 = UnpivotedOnlySelectedColumns.copy()
        RemovedColumns1 = AddedCustom4.drop(columns=["Index", "IndexShift_2", "variable", "ResProperty_2"])

        PivotedColumn1 = RemovedColumns1.pivot(index=['FO_ROW_ID','IndexShift_1'],columns='ResProperty_1',values='value').reset_index()

        FilteredRows01 = PivotedColumn1[(PivotedColumn1["ResWBS_Start"].notnull() & PivotedColumn1['ResWBS_Start'].str.len() > 0)]

        RemovedColumns2 = AddedCustom4.drop(columns=["Index", "IndexShift_1", "variable", "ResProperty_1"])
        PivotedColumn2 = RemovedColumns2.pivot(index=['FO_ROW_ID','IndexShift_2'],columns='ResProperty_2',values='value').reset_index()

        FilteredRows02 = PivotedColumn2[(PivotedColumn2["ResWBS_Start"].notnull() & PivotedColumn2['ResWBS_Start'].str.len() > 0)]

        columns = [
            'FO_ROW_ID', 
            'DATE_TIME_STAMP_End',
            'DATE_TIME_STAMP_Start', 
            'EVENT_ROW_ID_End', 
            'EVENT_ROW_ID_Start',
            'ResTk_End', 
            'ResTk_Start', 
            'ResWBS_End', 
            'ResWBS_Start', 
            'USER_ID_End',
            'USER_ID_Start'
        ]
        FilteredRows01 = FilteredRows01[columns]
        FilteredRows02 = FilteredRows01[columns]
        combined = pd.concat([FilteredRows01,FilteredRows02])

        combined = combined.rename(
            columns=
            {
                "ResTk_Start":"ResTk",
                "ResWBS_Start":"WBS",
                "EVENT_ROW_ID_Start": "EVENT_ROW_ID_Begin",
                "DATE_TIME_STAMP_Start": "DATE_TIME_STAMP_Begin"
            }
        )

        columns = [
            "FO_ROW_ID",
            "ResTk", 
            "WBS", 
            "EVENT_ROW_ID_Begin", 
            "EVENT_ROW_ID_End", 
            "DATE_TIME_STAMP_Begin", 
            "DATE_TIME_STAMP_End", 
            "USER_ID_Start", 
            "USER_ID_End"
        ]
        Tools_with_resersvations  = combined#[columns]

        final = combined[columns]
        return final
    except:
        return pd.DataFrame()

In [3]:
Fab300_raw_reservations = pd.read_csv("Fab300_raw_reservations.csv")
Fab300_raw_reservations.head()

,FO_ROW_ID,ResTk,ResWBS,EVENT_ROW_ID,USER_ID,DATE_TIME_STAMP
0,74,4155748,32414/32513,943475937,alcott43,10/24/2022 9:54
1,74,NaN,NaN,943902421,alcott43,10/25/2022 15:42
2,79,4155668,30990/32063,942950523,kramac15,10/21/2022 15:54
3,79,NaN,NaN,942961454,thukka30,10/21/2022 16:30
4,79,4157252,30990/32063,943831996,jagalu79,10/25/2022 10:55


In [43]:
def processFab300RawReservations(df):
    columns = [
        "FO_ROW_ID",
        "DATE_TIME_STAMP_Begin",
        "DATE_TIME_STAMP_End",
        'EVENT_ROW_ID_Begin', 
        'EVENT_ROW_ID_End',
        'ResTk',
        'USER_ID_Start',
        'USER_ID_End',
        'WBS'
    ]

    for_row_ids = Fab300_raw_reservations["FO_ROW_ID"].unique()

    Tools_with_reservations = pd.DataFrame(columns=columns)

    for row_id in for_row_ids:
        grpdata = Fab300_raw_reservations[Fab300_raw_reservations["FO_ROW_ID"] == row_id]

        df = identify_reservations(grpdata)
        if df.shape[0] > 0:
            print(row_id)
            print("------------------------------")
            print(df.head(5))
            Tools_with_reservations = pd.concat([Tools_with_reservations,df])


    return Tools_with_reservations

In [44]:
df = pd.read_csv("Fab300_raw_reservations.csv")
Tools_with_reservations = processFab300RawReservations(df)
Tools_with_reservations

736
------------------------------
ResProperty_1  FO_ROW_ID    ResTk          WBS EVENT_ROW_ID_Begin  \
2                    736  4152709  31499/00880          942682074   
3                    736  4155938  31499/00880          943409222   
4                    736  4155938  31499/00880          943410012   
2                    736  4152709  31499/00880          942682074   
3                    736  4155938  31499/00880          943409222   

ResProperty_1 EVENT_ROW_ID_End DATE_TIME_STAMP_Begin DATE_TIME_STAMP_End  \
2                    943409217   2022-10-20 18:19:00 2022-10-23 19:07:00   
3                    943409962   2022-10-23 19:08:00 2022-10-23 19:12:00   
4                    943471163   2022-10-23 19:12:00 2022-10-24 09:24:00   
2                    943409217   2022-10-20 18:19:00 2022-10-23 19:07:00   
3                    943409962   2022-10-23 19:08:00 2022-10-23 19:12:00   

ResProperty_1 USER_ID_Start USER_ID_End  
2                  groven85       stass  
3        

10648
------------------------------
ResProperty_1  FO_ROW_ID    ResTk          WBS EVENT_ROW_ID_Begin  \
1                  10648  4167464  31199/30117          945949030   
1                  10648  4167464  31199/30117          945949030   

ResProperty_1 EVENT_ROW_ID_End DATE_TIME_STAMP_Begin DATE_TIME_STAMP_End  \
1                    946023891   2022-11-02 19:18:00 2022-11-03 03:40:00   
1                    946023891   2022-11-02 19:18:00 2022-11-03 03:40:00   

ResProperty_1 USER_ID_Start USER_ID_End  
1                    kato15    irutha45  
1                    kato15    irutha45  
12107
------------------------------
ResProperty_1  FO_ROW_ID    ResTk          WBS EVENT_ROW_ID_Begin  \
1                  12107  4155179  36003/01224          942536288   
2                  12107  4155604  36003/01224          942862177   
3                  12107      N/a  36003/01224          943866617   
4                  12107  4162018  31199/01873          944175354   
5                 

7                     roy82       roy82  
20891
------------------------------
ResProperty_1  FO_ROW_ID    ResTk          WBS EVENT_ROW_ID_Begin  \
2                  20891  4160611  32005/01675          944185648   
3                  20891  4160617  32005/01675          944398915   
4                  20891  4166433  32005/01675          944843364   
5                  20891  4167252  32005/01675          945662482   
6                  20891  4167257  32005/01675          945776814   

ResProperty_1 EVENT_ROW_ID_End DATE_TIME_STAMP_Begin DATE_TIME_STAMP_End  \
2                    944208427   2022-10-26 13:41:00 2022-10-26 15:15:00   
3                    944436494   2022-10-27 07:44:00 2022-10-27 10:46:00   
4                    944957179   2022-10-28 14:09:00 2022-10-28 19:07:00   
5                    945666719   2022-11-01 10:06:00 2022-11-01 16:40:00   
6                    945798814   2022-11-02 09:26:00 2022-11-02 10:39:00   

ResProperty_1 USER_ID_Start USER_ID_End  
2      

,FO_ROW_ID,DATE_TIME_STAMP_Begin,DATE_TIME_STAMP_End,EVENT_ROW_ID_Begin,EVENT_ROW_ID_End,ResTk,USER_ID_Start,USER_ID_End,WBS
2,736,2022-10-20 18:19:00,2022-10-23 19:07:00,942682074,943409217,4152709,groven85,stass,31499/00880
3,736,2022-10-23 19:08:00,2022-10-23 19:12:00,943409222,943409962,4155938,stass,stass,31499/00880
4,736,2022-10-23 19:12:00,2022-10-24 09:24:00,943410012,943471163,4155938,stass,groven85,31499/00880
2,736,2022-10-20 18:19:00,2022-10-23 19:07:00,942682074,943409217,4152709,groven85,stass,31499/00880
3,736,2022-10-23 19:08:00,2022-10-23 19:12:00,943409222,943409962,4155938,stass,stass,31499/00880
...,...,...,...,...,...,...,...,...,...
2,24322,2022-10-23 07:47:00,2022-10-23 08:20:00,943317615,943323889,4155975,carola29,carola29,42199/00160
3,24322,2022-10-24 12:21:00,2022-10-24 17:26:00,943520298,943595945,4155735,drijboom,drijboom,42199/00160
4,24322,2022-10-24 17:26:00,2022-10-26 16:50:00,943595947,944236288,4155735,drijboom,scheer40,42199/00160
5,24322,2022-10-27 10:14:00,2022-10-28 09:47:00,944427413,944760571,4157501,drijboom,drijboom,42199/00160


In [5]:
Tools_Parents = pd.read_csv("Tools_Parents.csv")
Tools_Parents["CSIM_TIMESTAMP"] = pd.to_datetime(Tools_Parents["CSIM_TIMESTAMP"])
Tools_Parents

,DEL_FLAG,FACILITY,ENT_NAME,ROW_ID,CSIM_TIMESTAMP,Area
0,N,PLINE300,AIXTRON,74,2022-10-19 11:03:00,Area/DDEPI/Deposition/EPI
1,N,PLINE200,ARCHER,75,2022-10-20 17:45:00,Area/ILIME/Metrology/Inspections
2,N,PLINE300,DNS,76,2022-10-20 21:57:00,Area/WMCE/Clean/Clean
3,N,PLINE300,DUMMY300,78,2022-09-10 11:33:00,Area/Dummy-Area/Dummy-SubArea/Dummy-EntityGroup
4,N,PLINE300,F5_SCD,79,2022-10-20 20:51:00,Area/ILIME/Metrology/Ellipsometry
...,...,...,...,...,...,...
389,N,PLINE200,CHAMEO_UP,25772,2022-07-14 16:52:00,Area/WBA/Assembly/FlipChip
390,N,PLINE300,GT2000,26118,2022-10-14 11:38:00,Area/ILIME/Metrology/Inspections
391,N,PLINE200,UV_MANUAL,26135,2022-10-20 14:07:00,Area/WBA/Assembly/Dicing
392,N,PLINE200,A412_2,26167,2022-10-16 06:25:00,Area/DDDIFFUSION/Anneal/AnnealOven


In [27]:
def FAB300_with_tool_names(Tools_with_reservations,Tools_Parents):
    Expanded_Tools_parents = pd.merge(
        Tools_with_reservations, 
        Tools_Parents, 
        left_on=["FO_ROW_ID"], 
        right_on=["ROW_ID"], 
        how="left",
        suffixes=["","_y"]
    )
    Expanded_Tools_parents = Expanded_Tools_parents.rename(
        columns={
            "ENT_NAME":"Tool",
            "USER_ID_Start":"USER_ID_Begin",
            "USER_ID_End":"USER_ID_End"
        }
    )
    Expanded_Tools_parents_filteredRows = Expanded_Tools_parents[
        Expanded_Tools_parents["USER_ID_Begin"] == Expanded_Tools_parents["USER_ID_End"]
    ]
    Expanded_Tools_parents_filteredRows = Expanded_Tools_parents_filteredRows.rename(
        columns={
            "USER_ID_Begin":"User_id",
            "DATE_TIME_STAMP_Begin":"Begin",
            "DATE_TIME_STAMP_End":"End"
        }
    )
    Expanded_Tools_parents_filteredRows = Expanded_Tools_parents_filteredRows.sort_values(["FO_ROW_ID","EVENT_ROW_ID_Begin"])
    Fab300_Res_id = range(0,len(Expanded_Tools_parents_filteredRows))
    Expanded_Tools_parents_filteredRows["Fab300_Res_id"] = Fab300_Res_id
    Expanded_Tools_parents_filteredRows = Expanded_Tools_parents_filteredRows[
        (
            Expanded_Tools_parents_filteredRows["Tool"].notnull() & 
            Expanded_Tools_parents_filteredRows['Tool'].str.len() > 0)
    ]
    columns = [
        "FO_ROW_ID",
        "EVENT_ROW_ID_Begin",
        "Begin",
        "End",
        "Fab300_Res_id",
        "FACILITY",
        "ResTk",
        "Tool",
        "User_id",
        "WBS"
    ]
    Expanded_Tools_parents_filteredRows[columns]
    FAB300_with_tool_names = Expanded_Tools_parents_filteredRows[columns]

    #Sample for FO_ROW_ID == 76
    return FAB300_with_tool_names

In [28]:
Tools_Parents = pd.read_csv("Tools_Parents.csv")
Tools_Parents["CSIM_TIMESTAMP"] = pd.to_datetime(Tools_Parents["CSIM_TIMESTAMP"])

df_FAB300_with_tool_names = FAB300_with_tool_names(Tools_with_reservations,Tools_Parents)
df_FAB300_with_tool_names

,FO_ROW_ID,EVENT_ROW_ID_Begin,Begin,End,Fab300_Res_id,FACILITY,ResTk,Tool,User_id,WBS
1,736,943409222,2022-10-23 19:08:00,2022-10-23 19:12:00,0,PLINE300,4155938,EPSILON3200,stass,31499/00880
4,736,943409222,2022-10-23 19:08:00,2022-10-23 19:12:00,1,PLINE300,4155938,EPSILON3200,stass,31499/00880
6,4673,942498088,2022-10-20 06:21:00,2022-10-20 08:07:00,2,PLINE300,4155188,NOVA200_2,cochet,32299/00013
31,4673,942498088,2022-10-20 06:21:00,2022-10-20 08:07:00,3,PLINE300,4155188,NOVA200_2,cochet,32299/00013
9,4673,942848233,2022-10-21 10:28:00,2022-10-21 12:09:00,4,PLINE300,4155675,NOVA200_2,davide98,32414/32513
...,...,...,...,...,...,...,...,...,...,...
417,24322,943317615,2022-10-23 07:47:00,2022-10-23 08:20:00,283,PLINE300,4155975,HELIOS_G5,carola29,42199/00160
412,24322,943520298,2022-10-24 12:21:00,2022-10-24 17:26:00,284,PLINE300,4155735,HELIOS_G5,drijboom,42199/00160
418,24322,943520298,2022-10-24 12:21:00,2022-10-24 17:26:00,285,PLINE300,4155735,HELIOS_G5,drijboom,42199/00160
414,24322,944427413,2022-10-27 10:14:00,2022-10-28 09:47:00,286,PLINE300,4157501,HELIOS_G5,drijboom,42199/00160


In [34]:
#IIO_raw_reservations
def IIO_without_modules(df):
    df = df.reset_index()
    df["Begin"] = pd.to_datetime(df["Begin"],format="%d/%m/%Y %H:%M")
    df["End"] = pd.to_datetime(df["End"],format="%d/%m/%Y %H:%M")
    df = df.sort_values(["WBS","Tool","FACILITY","Module","Begin"])
    df["End_Down"] = df["End"].shift(1)
    df["Adjacent_Down"] = np.where(df['Begin']==df['End_Down'], True, False)
    df["IndexCopy"] = np.where(df["Adjacent_Down"] == True,np.nan,df["index"])
    df["IndexCopy2"] = df["IndexCopy"].fillna(method='ffill')
    df["Module"] = df["Module"].fillna("")
    df["Description"] = df["Description"].fillna("")
    df = df[['WBS','FACILITY', 'Module','Tool','Begin', 'Description', 'End', 'User_id','IndexCopy2','End_Down']]


    params = {
        'Begin': 'min',
        'End': 'max',
        'Description': lambda x: ';'.join(sorted(pd.Series.unique(x))),
        'User_id': lambda x: ';'.join(sorted(pd.Series.unique(x)))
    }
    sub = df[["WBS","Tool","FACILITY","Module","Begin","End","Description",'User_id','IndexCopy2']]
    GroupedRows1 = sub.groupby(["WBS","Tool","FACILITY","Module",'IndexCopy2']).agg(params).reset_index()

    params = {
        'Module': lambda x: ';'.join(sorted(pd.Series.unique(x))),
        'Description': 'first',
        'User_id': 'first'
    }
    

    GroupedRows1 = GroupedRows1.groupby(["WBS","Tool","FACILITY","Begin", "End"]).agg(params).reset_index()
    GroupedRows1 = GroupedRows1.sort_values(["FACILITY", "Tool", "WBS", "Begin", "End"])

    Index = range(0,len(GroupedRows1))
    GroupedRows1["IIO_Res_id"] = Index
    
    GroupedRows1 = GroupedRows1.rename(
        columns={
            "Module":"Modules"
        }
    )
    
    IIO_without_modules = GroupedRows1.copy()
    return IIO_without_modules

In [35]:
df = pd.read_csv("IIO_raw_reservations - test.csv")
df_IIO_without_modules = IIO_without_modules(df)
df_IIO_without_modules

,WBS,Tool,FACILITY,Begin,End,Modules,Description,User_id,IIO_Res_id
291,31012/01506,300MM_DICER,PLINE200,2022-09-29 09:00:00,2022-09-29 17:00:00,,lots on hold,schoofs,0
292,31012/01506,300MM_DICER,PLINE200,2022-10-14 13:30:00,2022-10-14 17:00:00,,lots on hold,schoofs,1
293,31012/01506,300MM_DICER,PLINE200,2022-10-18 13:30:00,2022-10-18 17:00:00,,lots on hold,schoofs,2
2499,36671/01224,300MM_DICER,PLINE200,2022-10-05 11:00:00,2022-10-05 12:00:00,,IN4MED lot,ameghn57,3
2500,36671/01224,300MM_DICER,PLINE200,2022-10-07 09:30:00,2022-10-07 12:00:00,,DEVINP LOTS,ameghn57,4
...,...,...,...,...,...,...,...,...,...
2356,36003/02120,ZETA,PLINE300,2022-09-30 13:30:00,2022-09-30 17:00:00,,Recipe setting,tuncaa41,3097
2357,36003/02120,ZETA,PLINE300,2022-10-07 09:30:00,2022-10-07 11:00:00,,Recipe practice,tuncaa41,3098
2358,36003/02120,ZETA,PLINE300,2022-10-17 12:30:00,2022-10-17 17:00:00,,Recipe setting,tuncaa41,3099
2359,36003/02120,ZETA,PLINE300,2022-10-24 10:00:00,2022-10-24 12:00:00,,recipe testing,tuncaa41,3100


In [40]:
Source_fab = df_FAB300_with_tool_names.copy()
RC_fab = Source_fab.drop(columns=["ResTk", "User_id"])
UnPivot_fab = pd.melt(RC_fab, id_vars=["WBS", "FACILITY", "Tool", "Fab300_Res_id"], 
                value_vars=["Begin", "End"])
UnPivot_fab = UnPivot_fab.rename(
columns={
    "variable": "FAB300_BeginEnd",
    "value": "DateTime"
})

Source_iio = df_IIO_without_modules
RC_iio = Source_iio.drop(columns=["Modules", "User_id", "Description"])
UnPivot_iio = pd.melt(RC_iio, id_vars=["WBS", "FACILITY", "Tool", "IIO_Res_id"], 
                value_vars=["Begin", "End"])

UnPivot_iio = UnPivot_iio.rename(
columns={
    "variable": "IIO_BeginEnd",
    "value": "DateTime"
})

fab_iio_together  = pd.concat([UnPivot_fab, UnPivot_iio], ignore_index=True)

In [41]:
def Fab300_iio_merger(df):
    df["DateTime"] = pd.to_datetime(df["DateTime"])
    df = df.sort_values(["WBS","FACILITY",'Tool',"DateTime","FAB300_BeginEnd"])

    df["Fab300_Res_id_UP"] = df["Fab300_Res_id"].fillna(method='bfill')
    df["Fab300_Res_id_DOWN"] = df["Fab300_Res_id"].fillna(method='ffill')
    df["IIO_Res_id_UP"] = df["IIO_Res_id"].fillna(method='bfill')
    df["IIO_Res_id_DOWN"] = df["IIO_Res_id"].fillna(method='ffill')

    fab_iio_Filtered_Rows = df[
        (df["Fab300_Res_id_UP"] == df["Fab300_Res_id_DOWN"])
        & (df["IIO_Res_id_UP"] == df["IIO_Res_id_DOWN"])
        & (df["Fab300_Res_id_UP"].notnull())
        & (df["IIO_Res_id_UP"].notnull())
    ] 
    
    Removed_Other_Columns = fab_iio_Filtered_Rows[["Fab300_Res_id_UP", "IIO_Res_id_UP"]]
    Removed_Duplicates = Removed_Other_Columns.drop_duplicates()
    Renamed_Columns = Removed_Duplicates.rename(
        columns = {
            "Fab300_Res_id_UP":"Fab300_Res_id",
            "IIO_Res_id_UP":"IIO_Res_id"
        }
    )

    index = range(0,len(Renamed_Columns))
    Renamed_Columns["Index"] = index
    Added_Index = Renamed_Columns.copy()
    State_DOWN_Remove = Added_Index.copy()
    State_DOWN_Remove.loc[-1] = [np.nan,np.nan,-1]  # adding a row
    State_DOWN_Insert = State_DOWN_Remove.copy()
    State_DOWN_Insert = State_DOWN_Insert.sort_values(["Index"])
    index2 = range(0,len(State_DOWN_Insert))
    State_DOWN_Insert["Index2"] = index2

    State_DOWN_Add_Index = State_DOWN_Insert.copy()
    State_DOWN_Rename = State_DOWN_Add_Index.rename(
        columns = {
            "Fab300_Res_id":"Fab300_Res_id_DOWN",
            "IIO_Res_id":"IIO_Res_id_DOWN"
        }
    )

    With_DOWN = Added_Index.merge(
        State_DOWN_Rename, 
        left_on=["Index"],
        right_on=["Index2"],
        suffixes=["","_y"],
        how = 'left'
    )

    With_DOWN["Index3"] = np.where(
        (With_DOWN['Fab300_Res_id']==With_DOWN['Fab300_Res_id_DOWN'])
        | (With_DOWN['IIO_Res_id']==With_DOWN['IIO_Res_id_DOWN']), 
        With_DOWN['Index'], 
        np.nan
    )


    Replaced_Value = With_DOWN.copy()
    Replaced_Value["Index4"] = Replaced_Value["Index3"].fillna(method='ffill')
    Replaced_Value = Replaced_Value.drop(columns=["Fab300_Res_id_DOWN", "IIO_Res_id_DOWN"])
    Renamed_Columns1 = Replaced_Value.rename(
    columns={
        "Index4":"Cluster"
    })


    Merged_queries = Renamed_Columns1.merge(
        Source_fab, 
        left_on=["Fab300_Res_id"],
        right_on=["Fab300_Res_id"],
        suffixes=["","_y"],
        how = 'left'
    )
    Expanded_Fab300_with_tool_names = Merged_queries.rename(
    columns ={
        "Begin": "Fab300_Begin", 
        "End": "Fab300_End", 
        "User_id": "Fab300_User_id"
    })
    Merged_queries_1 = Expanded_Fab300_with_tool_names.merge(
        Source_iio, 
        left_on=["IIO_Res_id"],
        right_on=["IIO_Res_id"],
        suffixes=["","_y"],
        how = 'left'
    )
    Expanded_IIO_without_modules = Merged_queries_1.rename(
    columns ={
        "Begin": "IIO_Begin", 
        "End": "IIO_End", 
        "Modules": "Modules",
        "User_id": "IIO_User_id",
        "Description": "Description"
    })
    Expanded_IIO_without_modules = Expanded_IIO_without_modules.drop(
        columns=["FACILITY_y","Modules","Tool_y","IIO_User_id","WBS_y",'Index', 'Index_y', 'Index2', 'Index3']
    )
    
    return Expanded_IIO_without_modules

In [42]:
columns = [
    'Fab300_Res_id', 
    'IIO_Res_id', 
    'Cluster', 
    'Fab300_Begin', 
    'Fab300_End',
    'FACILITY', 
    'ResTk', 
    'Tool', 
    'Fab300_User_id', 
    'WBS', 
    'IIO_Begin',
    'Description', 
    'IIO_End'
]

Fab300_IIO_overlaps_ids = pd.DataFrame(columns = columns)

WBSs = fab_iio_together["WBS"].unique()

for wbs in WBSs:
    wbsdata = fab_iio_together[
        (fab_iio_together["WBS"] == wbs)
    ]
    facilities = wbsdata["FACILITY"].unique()
    for facility in facilities:
        facilityData = wbsdata[
            (wbsdata["FACILITY"] == facility)
        ]
        
        tools = facilityData["Tool"].unique()
        for tool in tools:
            wbstooldata = facilityData[
                (wbsdata["Tool"] == tool)
            ]
            df = Fab300_iio_merger(wbstooldata)
            Fab300_IIO_overlaps_ids = pd.concat([Fab300_IIO_overlaps_ids,df])
            
            
Fab300_IIO_overlaps_ids

C:\Users\FRANCI~1\AppData\Local\Temp/ipykernel_12852/1189357799.py:33: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  wbstooldata = facilityData[
C:\Users\FRANCI~1\AppData\Local\Temp/ipykernel_12852/1812784099.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["DateTime"] = pd.to_datetime(df["DateTime"])
C:\Users\FRANCI~1\AppData\Local\Temp/ipykernel_12852/1812784099.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["DateTime"] = pd.to_datetime(df["DateTime"])
C:\User

,Fab300_Res_id,IIO_Res_id,Cluster,Fab300_Begin,Fab300_End,FACILITY,ResTk,Tool,Fab300_User_id,WBS,IIO_Begin,Description,IIO_End,FO_ROW_ID,EVENT_ROW_ID_Begin
0,2.0,1850.0,NaN,2022-10-20 06:21:00,2022-10-20 08:07:00,PLINE300,4155188,NOVA200_2,cochet,32299/00013,2022-10-20 06:00:00,BSD,2022-10-20 08:00:00,4673,942498088
1,3.0,1850.0,1.0,2022-10-20 06:21:00,2022-10-20 08:07:00,PLINE300,4155188,NOVA200_2,cochet,32299/00013,2022-10-20 06:00:00,BSD,2022-10-20 08:00:00,4673,942498088
2,6.0,1852.0,1.0,2022-10-21 12:32:00,2022-10-21 14:29:00,PLINE300,4155541,NOVA200_2,beirnaer,32299/00013,2022-10-21 12:30:00,sem,2022-10-21 13:00:00,4673,942882222
3,7.0,1852.0,3.0,2022-10-21 12:32:00,2022-10-21 14:29:00,PLINE300,4155541,NOVA200_2,beirnaer,32299/00013,2022-10-21 12:30:00,sem,2022-10-21 13:00:00,4673,942882222
4,8.0,1853.0,3.0,2022-10-25 11:46:00,2022-10-25 13:20:00,PLINE300,4155417,NOVA200_2,cochet,32299/00013,2022-10-25 11:00:00,BSD,2022-10-25 13:30:00,4673,943844381
5,9.0,1853.0,5.0,2022-10-25 11:46:00,2022-10-25 13:20:00,PLINE300,4155417,NOVA200_2,cochet,32299/00013,2022-10-25 11:00:00,BSD,2022-10-25 13:30:00,4673,943844381
6,10.0,1854.0,5.0,2022-10-26 09:48:00,2022-10-26 14:14:00,PLINE300,4160607,NOVA200_2,cochet,32299/00013,2022-10-26 09:30:00,BSD,2022-10-26 12:30:00,4673,944129182
7,11.0,1854.0,7.0,2022-10-26 09:48:00,2022-10-26 14:14:00,PLINE300,4160607,NOVA200_2,cochet,32299/00013,2022-10-26 09:30:00,BSD,2022-10-26 12:30:00,4673,944129182
0,50.0,2336.0,NaN,2022-10-20 09:15:00,2022-10-20 09:50:00,PLINE300,4151413,SEMVISION,beirnaer,32299/00013,2022-10-20 08:00:00,sem,2022-10-20 10:00:00,4926,942522027
1,51.0,2336.0,1.0,2022-10-20 09:15:00,2022-10-20 09:50:00,PLINE300,4151413,SEMVISION,beirnaer,32299/00013,2022-10-20 08:00:00,sem,2022-10-20 10:00:00,4926,942522027
